In [1]:
# 라이브러리 불러오기
from selenium import webdriver 
import requests
from bs4 import BeautifulSoup as BS
import pandas as pd
import time
import re
import pickle
from selenium.webdriver.common.by import By
import datetime

In [2]:
# 크롬 드라이버 로드
file_path = "C:\\Users\\cho03\\Downloads\\세미나\\방학 세미나\\"
path = file_path + 'chromedriver.exe'
driver = webdriver.Chrome(path)

C:\Users\cho03\AppData\Local\Temp\ipykernel_11220\388304711.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


In [3]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36"
}

In [4]:
def press(url):
    response = requests.get(url, headers = headers)
    html_content = response.text
    soup = BS(html_content, 'html.parser')
    
    find_genre = soup.find('p',class_='c_text')
    text_content = find_genre.text

    start_index = text_content.find('ⓒ ')
    end_index = text_content.find('.')
    
    if start_index != -1:
        extracted_text = text_content[start_index + len('ⓒ '):end_index]
        ch = extracted_text
    return ch

In [5]:
# 네이버 블로그 검색 페이지로 이동
driver.get('https://search.naver.com/search.naver?where=news&query=%EB%B6%80%EB%AA%A8%20%EB%B6%80%EC%96%91&sm=tab_opt&sort=0&photo=0&field=0&pd=3&ds=2021.07.19&de=2023.07.19&docid=&related=0&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so%3Ar%2Cp%3Afrom20210719to20230719&is_sug_officeid=0')

In [6]:
title = []
url = []
doc = []
ch = []

for j in range(400): 
    try:
        a = driver.find_elements(By.CSS_SELECTOR, 'a.info:not(.press)')
        try:
            for i in range(len(a)):
                html = driver.page_source
                soup = BS(html, 'html.parser')
                a[i].click()
                time.sleep(1)
                driver.switch_to.window(driver.window_handles[1])

                if('news.naver.com' not in driver.current_url) :
                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])
                    continue

                b = driver.find_element(By.CSS_SELECTOR, "#title_area > span").text
                c = driver.current_url
                d = driver.find_element(By.CSS_SELECTOR, "#newsct_article").text
                e = press(c)

                title.append(b)
                url.append(c)
                doc.append(d)
                ch.append(e)

                driver.close()
                driver.switch_to.window(driver.window_handles[0])

            driver.find_element(By.CLASS_NAME, 'btn_next').click()
            print(j+1)
        except:
            continue
    except:
        continue

In [13]:
# 크롤링한 데이터를 데이터프레임으로 변환
raw_data = pd.DataFrame()
raw_data['title'] = title
raw_data['url'] = url
raw_data['doc'] = doc
raw_data['ch'] = ch

In [14]:
# 데이터프레임을 pickle파일로 저장
f = open(file_path + "news.pkl", "wb")
pickle.dump(raw_data,f)

f.close()

In [9]:
# 크롬 드라이버 종료
driver.quit()

In [ ]:
# 저장된 pickle 파일을 불러옴
f = open(file_path + "news.pkl", "rb")
temp_file = pickle.load(f)
f.close()

temp_file